# EOF Animation

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import animatplot as amp
import matplotlib.colors as colors

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar, ricker, cwt, ellip, csd
from scipy.interpolate import griddata
from scipy.spatial import Voronoi, voronoi_plot_2d

from matplotlib import animation
from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'
# %matplotlib notebook

## Import and clean up the data

In [2]:
# Import data
# All data (hour resolution)
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

In [3]:
# Individual stations (minuite resolution)
DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

DE = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DiscoveryElementary_temperature.dat', 
                 header=2)

HL = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Helgesen_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

JM = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JohnMuir_temperature.dat', 
                 header=2)

KT = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Keating_temperature.dat', 
                 header=2)

US = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

# Coastline 
MAP = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DataCoast.dat', 
                 sep='\s+', names = ["Lon", "Lat"])

### Clean up the All Stations data

In [4]:
# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][1]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][1]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4

In [5]:
# De-Nan
AS_DN = AS.dropna(axis=0)

### Clean up the minute resolution data

In [6]:
def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

In [7]:
DataM = [DC, DE, HL, JB, JM, KT, US]
DataMLabels = ['Deep Cove', 'Discovery', 'Helgesen', 'James Bay', 'John Muir', 'Keating', 'UVicSci']

In [8]:
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

In [9]:
Mn = {
    "DC" : DataMIns[0].rename(columns={"Temperature": "DC"}),
    "DE" : DataMIns[1].rename(columns={"Temperature": "DE"}),
    "HL" : DataMIns[2].rename(columns={"Temperature": "HL"}),
    "JB" : DataMIns[3].rename(columns={"Temperature": "JB"}),
    "JM" : DataMIns[4].rename(columns={"Temperature": "JM"}),
    "KT" : DataMIns[5].rename(columns={"Temperature": "KT"}),
    "US" : DataMIns[6].rename(columns={"Temperature": "US"})
}

In [10]:
# De-Nan
Mn_DN  = {
    "DC" : Mn["DC"].dropna(axis=0),
    "DE" : Mn["DE"].dropna(axis=0),
    "HL" : Mn["HL"].dropna(axis=0),
    "JB" : Mn["JB"].dropna(axis=0),
    "JM" : Mn["JM"].dropna(axis=0),
    "KT" : Mn["KT"].dropna(axis=0),
    "US" : Mn["US"].dropna(axis=0)
}

In [11]:
# Matrix of all the minute resolution data
MS = pd.concat([Mn['DC'], Mn['DE'], Mn['HL'], Mn['JB'], Mn['JM'], Mn['KT'], Mn['US']], axis=1, sort=True)

### Get station locations

In [12]:
# Station locations
StationLoc = list(AS)
Lon = np.array([float(list(AS)[i][1]) for i in range(1, np.shape(list(AS))[0])])
Lat = np.array([float(list(AS)[i][0]) for i in range(1, np.shape(list(AS))[0])])

## Create meshgrid for interpolation

In [13]:
step_size = 0.001
x = np.arange(np.min(MAP['Lon']), np.max(MAP['Lon']), step_size)
y = np.arange(np.min(MAP['Lat']), np.max(MAP['Lat']), step_size)

X, Y = np.meshgrid(x, y)

# Get the locations of the data
AS_loc = np.array([[float(AS.columns.values[i][1]), float(AS.columns.values[i][0])] for i in range(len(AS.columns.values))])

## Useful functions

### Maps and dimensions 

In [14]:
# Get dimensions 
def get_dim(x=X, y=Y):
    dim = [np.min(x), np.max(x), np.min(y), np.max(y)]
    Dim = [float(dim[n]) for n in range(len(dim))]
    return Dim

In [15]:
# Draw the map
def map(title='WHAT\'S THE THE TITLE YOU DONKEY?!?!?', size=(10, 10), land='#FFFFFF', water='#FFFFFF'):
    fig, ax = plt.subplots(1, 1, figsize=size)
    
    AddPoints = pd.DataFrame([[MAP['Lon'].min()-0.1, MAP['Lat'].min()-0.1], [MAP['Lon'].max()+0.1, MAP['Lat'].min()-0.1], [MAP['Lon'].max()+0.1, MAP['Lat'].max()+0.1]], columns=['Lon', 'Lat'])
    MAPIn = MAP.append(AddPoints, ignore_index=True)
    
    ax.fill(MAP['Lon'], MAP['Lat'], land, zorder=0, label='')
    ax.fill(MAPIn['Lon'], MAPIn['Lat'], water, zorder=2, label='')
    ax.plot(MAP['Lon'], MAP['Lat'], linewidth=np.min(size)/12, color='k', zorder=3, label='')

    ax.xaxis.set_major_formatter(StrMethodFormatter(r'{x:.1f}$^\circ$W'))
    ax.yaxis.set_major_formatter(StrMethodFormatter(r'{x:.2f}$^\circ$N'))
    mp.rc('xtick', labelsize=np.min(size)*1.7) 
    mp.rc('ytick', labelsize=np.min(size)*1.7) 
    plt.xticks(rotation=0)
    
    # Old and depricated string format:
    # Map2.xaxis.set_major_formatter(FormatStrFormatter(r'%1.1f$^\circ$W'))
    # Map2.yaxis.set_major_formatter(FormatStrFormatter(r'%1.2f$^\circ$N'))

    ax.set_xlim(-123.8, -123.25)
    ax.set_ylim(48.325, 48.725)
#     ax.grid(dashes=(1,1), color='#555555', zorder=0, linewidth=size[0]/6)
    ax.set_facecolor(water)
    
    ax.set_aspect('equal')
    maptitle = title
    ax.set_title("\n".join(wrap(maptitle, 50)), fontsize=np.min(size)*2.25)
    ax.set_xlabel(r'Longitude', fontsize=np.min(size)*2)
    ax.set_ylabel(r'Latitude', fontsize=np.min(size)*2)

### Heatmaps

In [16]:
# Heatmap for...um...map?
def heatmap(Array, index=0,  D=get_dim(), colourmap='coolwarm', size=(10, 8), title='TITLE DONKEY!!!', 
            units=r'Temperature difference from mean [$^\circ$C]', interp='none', normalize=False):
    
    map(title, size, land='#FFFFFF', water='#EEEEEE')
    plt.scatter(Lon, Lat, marker='o', s=5, color='k', zorder=1)
    Array_plot = plt.imshow(Array[index], extent=D, origin='lower', cmap=colourmap, interpolation=interp)
    cb = plt.colorbar(Array_plot, shrink=0.7, aspect=20, format='%.3f', label=units)
    cb.set_label(label=units, fontsize=np.min(size)*2)
    if normalize==True:
        plt.clim(np.nanmin(Array), np.nanmax(Array))
#     plt.clim(-15, 40)

In [17]:
# Emperical orthogonal functions
def EOF(Array=AS_DN, int_type='cubic', locations=AS_loc):
    # The covariance matrix
    mean = Array.mean(axis=1)
#     Array_DM = Array.sub(Array.mean(axis=1), axis=0)
#     Array2 = Array.interpolate(method='cubic', axis=0)
    Array_DM = Array.subtract(Array.mean())
    Array_DN = Array_DM.fillna(0)
    Array_CM = Array_DN.cov()

    # Eigenvalue and Eigenvector matrix
    Array_CM_Eval, Array_CM_Evec = np.linalg.eig(Array_CM)
    Array_CM_EvalM = np.identity(len(Array_CM_Eval)) * Array_CM_Eval
    
    # Normalize them
#     Array_CM_Evec = Array_CM_Evec/sum(abs(np.amax(Array_CM_Evec, axis=1))) # No need to normalise as numpy does it automatically
    Array_CM_Eval = Array_CM_Eval/sum(Array_CM_Eval)
    
    # Get the EOFs
    EOFs = np.array([griddata(locations, Array_CM_Evec[:,i], (X, Y), method=int_type) for i in range(len(Array_CM_Eval))])
    return EOFs, Array_CM_Evec, Array_CM_Eval

## Plot the empirical orthogonal functions 

In [18]:
EOFArray = AS.loc['01-May-2012':'31-Aug-2012']
EOFs, Evec, Eval = EOF(Array=EOFArray.resample('h').mean(), int_type='cubic', locations=AS_loc)

In [ ]:
for i in range(4):
    hm1 = heatmap(EOFs, index=i, title=r'EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[i]), interp='none', normalize=False)
    i+=1

<Figure size 720x576 with 2 Axes>

In [ ]:
plt.figure(figsize=10, 8)

heatmap(EOFs, index=i, title=r'EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[i]), interp='none', normalize=False)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(EOFArray.fillna(0)@Evec[0])
plt.plot(EOFArray.fillna(0)@Evec[1])
plt.plot(EOFArray.fillna(0)@Evec[2])
plt.plot(EOFArray.fillna(0)@Evec[3])
plt.tick_params(axis='both', which='major', labelsize=10, rotation=90)
# np.sqrt(Evec[0].dot(Evec[0]))

In [ ]:
alpha = np.outer(EOFArray.fillna(0)@Evec[0], Evec[0])

In [ ]:
np.shape(EOFArray.fillna(0)@Evec[0]), np.shape(Evec[0]), np.shape(alpha)

In [ ]:
alpha_H = np.array([griddata(AS_loc, alpha[i], (X, Y), method='cubic') for i in range(len(np.array(alpha)))])

In [ ]:
for i in range(4):
    hm1 = heatmap(EOFs, index=i, title=r'EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[i]), interp='none', normalize=False)
    i+=1